## Simple XGBoost tutorial


XGBoost (eXtreme Gradient Boosting) is a machine learning algorithm that uses an ensemble of decision trees and gradient boosting to make predictions.
It is very powerful and scalable, frequently among the best methods you can use in machine learning competitions.

### Brief description
XGBoost relies on ensemble learning via boosting, meaning that its purpose is to combine the predictive power of an arbitrary number of models. The result is one model whose output is an aggregation of the output of all (base) models. The base models can be of the same type or not. 

Bagging and Boosting are common methods to build ensemble learners but they differ in some crucial aspecs.

#### Bagging (bootstrap aggregating)
Decision trees are one of the most easily interpretable models but they exhibit highly variable behavior. To stabilize them a common solution is to combine bootstrap (training a model on a random sample of the data) with the aggregation of the outcome of more models each trainied on a different random sample. An example of bagging method is Random Forest.

#### Boosting
In boosting, the trees are built sequentially such that each subsequent tree aims to reduce the errors of the previous tree. Each tree learns from its predecessors and updates the residual errors. Hence, the tree that grows next in the sequence will learn from an updated version of the residuals.

The base learners in boosting are weak learners, the final learner is much stronger and brings down both the bias and the variance.

In contrast to bagging techniques like Random Forest, in which trees are grown to their maximum extent, boosting makes use of trees with fewer splits. Having a large number of trees might lead to overfitting.


#### Usefulness of XGBoost
- Very effective
- Supports many tasks: prediction, classification, ranking
- Flexible and able to handle data with a large number of features
- Scalable and [parallelizable](https://www.nvidia.com/en-us/glossary/data-science/xgboost/)


## How to use XGBoost in RecSys?

## Hint: reranking and hybridization

### Steps:
* Run your best algorithm and select a number of recommendations higher than the target cutoff, for example if you have to compute MAP@10, get 20 recommendations
* Build a dataframe whose samples are the user-item recommendations
* Add for each interaction some content features: item features, user features
* Add for each interaction some features derived by other algorithms: CBF prediction, hybrid prediction
* Add for each interaction other miscellaneous information: profile length, item popularity ..

## Dummy example

In [1]:
import os
os.chdir("/Users/gab/Documents/RecSysLocal1/newslim/recsys")

In [2]:
from typing import Tuple, Callable, Dict, Optional, List
import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

from Evaluation.Evaluator import EvaluatorHoldout



In [3]:
URM = pd.read_csv("./data/data_train.csv")
ICM = pd.read_csv("./data/data_ICM_metadata.csv")

In [4]:
unique_users = URM.user_id.unique()
max_user_id = unique_users.max()
min_user_id = unique_users.min()
num_users = unique_users.size
print(f"Max User ID: {max_user_id}, Min User ID: {min_user_id}, Number of IDs: {num_users}")

unique_items = ICM.item_id.unique()
max_item_id = ICM.item_id.max()
min_item_id = ICM.item_id.min()
num_items = unique_items.size
unique_features = ICM.feature_id.unique()
num_features = unique_features.size
min_features_id = ICM.feature_id.min()
max_features_id = ICM.feature_id.max()
print(f"Max Item ID: {max_item_id}, Min Item ID: {min_item_id}, Number of IDs: {num_items}")
print(f"Max Feature ID: {max_features_id}, Min Item ID: {min_features_id}, Number of IDs: {num_features}")

Max User ID: 35735, Min User ID: 0, Number of IDs: 35736
Max Item ID: 38120, Min Item ID: 0, Number of IDs: 38121
Max Feature ID: 94330, Min Item ID: 0, Number of IDs: 94331


In [5]:
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
import numpy as np

def dataset_splits(ratings, num_users, num_items, validation_percentage: float, testing_percentage: float, xgboost_split_percentage: float):
    seed = 1234

    # Construct the whole URM as a sparse matrix
    urm_all = sp.csr_matrix((ratings.data, (ratings.user_id, ratings.item_id)),
                            shape=(num_users, num_items))

    # Split into train + validation and test sets
    train_val_indices, test_indices = train_test_split(
        np.arange(len(ratings)),
        test_size=testing_percentage,
        shuffle=True,
        random_state=seed
    )
    
    # Split train + validation into train and validation
    train_indices, val_indices = train_test_split(
        train_val_indices,
        test_size=validation_percentage / (1 - testing_percentage),
        shuffle=True,
        random_state=seed
    )
    
    # Split the training set into `urm_rec_train` and `urm_XG_train`
    rec_train_indices, xg_train_indices = train_test_split(
        train_indices,
        test_size=xgboost_split_percentage,
        shuffle=True,
        random_state=seed
    )
    
    # Get user, item, and rating data for each set
    rec_train_data = ratings.iloc[rec_train_indices]
    xg_train_data = ratings.iloc[xg_train_indices]
    val_data = ratings.iloc[val_indices]
    test_data = ratings.iloc[test_indices]
    
    # Construct sparse matrices
    urm_rec_train = sp.csr_matrix((rec_train_data.data, (rec_train_data.user_id, rec_train_data.item_id)), 
                                  shape=(num_users, num_items))
    urm_XG_train = sp.csr_matrix((xg_train_data.data, (xg_train_data.user_id, xg_train_data.item_id)), 
                                 shape=(num_users, num_items))
    urm_validation = sp.csr_matrix((val_data.data, (val_data.user_id, val_data.item_id)), 
                                   shape=(num_users, num_items))
    urm_test = sp.csr_matrix((test_data.data, (test_data.user_id, test_data.item_id)), 
                             shape=(num_users, num_items))

    return urm_all, urm_rec_train, urm_XG_train, urm_validation, urm_test


In [9]:
# Call the function with appropriate split percentages
urm_all, urm_rec_train, urm_XG_train, urm_validation, urm_test = dataset_splits(
    URM, 
    num_users=num_users, 
    num_items=num_items, 
    validation_percentage=0.15, 
    testing_percentage=0.01,
    xgboost_split_percentage=0.3
)

icm_matrix = sp.csr_matrix((ICM.data, (ICM.item_id, ICM.feature_id)), 
                           shape=(num_items, num_features))

In [10]:
# # DON'T RUN

# from Data_manager.split_functions.split_train_validation_random_holdout import (
#     split_train_in_two_percentage_global_sample,
# )
# from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

# URM_all = pd.read_csv("data/data_train.csv")

# urm_rec_train, URM_test = split_train_in_two_percentage_global_sample(
#     URM_all, train_percentage=0.8
# )
# urm_rec_train, URM_validation = split_train_in_two_percentage_global_sample(
#     urm_rec_train, train_percentage=0.8
# )

# #ICM_genres = data_loaded.get_loaded_ICM_dict()["ICM_genres"]

### Let's select the recommendations to use to train XGBoost

Several options are possible:
- Take the best recommender and select, say, the top-100 recommended items
- Take multiple recommenders and use the union of their recommendations
- ...

In this example a single model will be used to select the candidate user-item interactions to use for XGBoost

In [11]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

candidate_generator_recommender = SLIMElasticNetRecommender(urm_rec_train)
candidate_generator_recommender.fit(topK= 1000, l1_ratio=0.003266986021483405, alpha=0.001, positive_only= True, do_feature_selection=True)

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 38121 (100.0%), 1119.91 column/sec. Elapsed time 34.04 sec
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 1.43 min. Items per second: 444.47


In [12]:
from scipy.sparse import save_npz

# Save the trained model's similarity matrix
save_npz("slimEN_weights_train.npz", candidate_generator_recommender.W_sparse)


In [13]:
from scipy.sparse import load_npz

# Load the similarity matrix
loaded_W_sparse = load_npz("slimEN_weights_train.npz")

# Assign it back to a SLIM_BPR model if needed
candidate_generator_recommender.W_sparse = loaded_W_sparse


## Build the dataframe with the predictions

Creating dataframes can be quite computationally expensive. Never use "append" or iteratively concatenate new elements when you have to run a significant number of iterations, it will take forever. Initialize instead the dimension you want.

In [14]:
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sps
import numpy as np
from xgboost import XGBRanker

n_users, n_items = urm_rec_train.shape

training_dataframe = pd.DataFrame(index=range(0, n_users), columns=["ItemID"])
training_dataframe.index.name = "UserID"

In [15]:
training_dataframe

,ItemID
UserID,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
35731,NaN
35732,NaN
35733,NaN


In [16]:
cutoff = 30

for user_id in tqdm(range(n_users)):
    recommendations = candidate_generator_recommender.recommend(user_id, cutoff=cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations

100%|██████████| 35736/35736 [00:19<00:00, 1844.93it/s]


In [17]:
training_dataframe

,ItemID
UserID,
0,"[6380, 572, 15454, 3074, 8569, 3146, 22936, 10..."
1,"[11148, 471, 3058, 14888, 11950, 516, 14907, 5..."
2,"[15542, 31260, 22471, 29218, 22692, 22421, 133..."
3,"[4878, 25140, 25643, 25079, 9742, 4299, 27392,..."
4,"[9812, 12632, 8505, 4348, 15902, 1075, 15702, ..."
...,...
35731,"[28246, 27741, 28332, 37739, 38027, 37277, 359..."
35732,"[27626, 36682, 36701, 37317, 28292, 37146, 374..."
35733,"[28304, 37853, 36458, 37579, 37950, 37177, 377..."


### Let's transform the dataframe so that we have one recommendation per row

In [18]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

,ItemID
UserID,
0,6380
0,572
0,15454
0,3074
0,8569
...,...
35735,36780
35735,37698
35735,36772


### We should add the target column, which is the correct recommendation as contained in the validation data

In [19]:
URM_validation_coo = sps.coo_matrix(urm_XG_train)

correct_recommendations = pd.DataFrame(
    {"UserID": URM_validation_coo.row, "ItemID": URM_validation_coo.col}
)
correct_recommendations

,UserID,ItemID
0,0,1015
1,0,2548
2,0,2600
3,0,2906
4,0,2920
...,...,...
444676,35735,37757
444677,35735,37800
444678,35735,37803
444679,35735,37805


In [20]:
training_dataframe = pd.merge(
    training_dataframe,
    correct_recommendations,
    on=["UserID", "ItemID"],
    how="left",
    indicator="Exist",
)
training_dataframe

,UserID,ItemID,Exist
0,0,6380,left_only
1,0,572,left_only
2,0,15454,left_only
3,0,3074,left_only
4,0,8569,left_only
...,...,...,...
1072075,35735,36780,left_only
1072076,35735,37698,left_only
1072077,35735,36772,left_only
1072078,35735,36164,left_only


In [21]:
# Count occurrences of each category in the "Exist" column
counts = training_dataframe["Exist"].value_counts()

print(counts)


Exist
left_only     1004838
both            67242
right_only          0
Name: count, dtype: int64


In [22]:
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns=["Exist"], inplace=True)
training_dataframe

,UserID,ItemID,Label
0,0,6380,False
1,0,572,False
2,0,15454,False
3,0,3074,False
4,0,8569,False
...,...,...,...
1072075,35735,36780,False
1072076,35735,37698,False
1072077,35735,36772,False
1072078,35735,36164,False


## Now let's add some features. For example, the prediction of other algorithms

This may take some time so it is a good idea to save this data and load it instead of calculating the scores every time.

In [23]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommenderICM import RP3betaRecommenderICM
from Recommenders.GabsHybridTesting.HybridRP3KNN import HybridRP3betaKNNRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

knn_cf = ItemKNNCFRecommender(urm_all)
knn_cf.fit(
    topK=10,
    shrink=264,
    similarity="jaccard",  
    normalize=False,
    feature_weighting="BM25", # or "TF-IDF" or "none"
    asymmetric_alpha =1.2043488778583
)

# Initialize with both URM and ICM
rp3beta_icm = RP3betaRecommenderICM(urm_all, icm_matrix)
rp3beta_icm.fit(
    topK= 45,
    alpha= 1.8233706541466446,
    beta= 0.3532994315100997,
    delta= 0.22907169615669423,
    min_rating= 0.9940576605024676,
    implicit= True,
    normalize_similarity= True
)

hybrid_recommender = HybridRP3betaKNNRecommender(
    URM_train=urm_all,
    rp3beta_icm_recommender=rp3beta_icm,
    knn_recommender=knn_cf,
    verbose=True
)
hybrid_recommender.fit(alpha=0.3053446904625561, normalize_similarity=True)

slim_recommender = SLIMElasticNetRecommender(urm_all)
slim_recommender.fit(topK= 1000, l1_ratio=0.003266986021483405, alpha=0.001, positive_only= True, do_feature_selection=True)


other_algorithms = {
    "ItemKNN": knn_cf,
    "RP3betaICM": rp3beta_icm,
    "Hybrid": hybrid_recommender,
    "SLIM_EN": slim_recommender
}

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 38121 (100.0%), 1028.80 column/sec. Elapsed time 37.05 sec
RP3betaRecommenderICM: Normalized URM and ICM
RP3betaRecommenderICM: Processed 38121 ( 100.00% ) in 2.62 min. Items per second: 242
RP3betaRecommenderICM: Computation completed
HybridRP3betaKNNRecommender: Initialized
HybridRP3betaKNNRecommender: Combining matrices with alpha=0.3053446904625561
HybridRP3betaKNNRecommender: Computation completed
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 38121 (100.0%), 1071.73 column/sec. Elapsed time 35.57 sec
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 1.63 min. Items per second: 390.27


In [24]:
training_dataframe = training_dataframe.set_index("UserID")

for user_id in tqdm(range(n_users)):
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()

        all_item_scores = rec_instance._compute_item_score(
            [user_id], items_to_compute=item_list
        )

        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list]

100%|██████████| 35736/35736 [01:17<00:00, 461.47it/s]


In [25]:
from sklearn.preprocessing import MinMaxScaler
# Normalize scores using Min-Max normalization
for rec_label in other_algorithms.keys():
    # Apply Min-Max normalization to the column
    scaler = MinMaxScaler()
    training_dataframe[rec_label] = scaler.fit_transform(
        training_dataframe[[rec_label]]
    )

# Reset the index and rename the columns
training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns={"index": "UserID"})

In [26]:
training_dataframe

,UserID,ItemID,Label,ItemKNN,RP3betaICM,Hybrid,SLIM_EN
0,0,6380,False,0.090662,0.085996,0.094839,0.223108
1,0,572,False,0.101362,0.138505,0.181966,0.209152
2,0,15454,False,0.029662,0.051473,0.057410,0.155842
3,0,3074,False,0.017430,0.100254,0.064270,0.150944
4,0,8569,False,0.000000,0.059708,0.061508,0.148979
...,...,...,...,...,...,...,...
1072075,35735,36780,False,0.000000,0.057894,0.053736,0.100020
1072076,35735,37698,False,0.116613,0.000000,0.045100,0.098611
1072077,35735,36772,False,0.045117,0.031387,0.037053,0.095042
1072078,35735,36164,False,0.058774,0.011310,0.030548,0.091382


## The popularity of the item, the user activity etc..

In [27]:
item_popularity = np.ediff1d(sps.csc_matrix(urm_rec_train).indptr)

training_dataframe["item_popularity"] = item_popularity[
    training_dataframe["ItemID"].values.astype(int)
]
training_dataframe

,UserID,ItemID,Label,ItemKNN,RP3betaICM,Hybrid,SLIM_EN,item_popularity
0,0,6380,False,0.090662,0.085996,0.094839,0.223108,162
1,0,572,False,0.101362,0.138505,0.181966,0.209152,56
2,0,15454,False,0.029662,0.051473,0.057410,0.155842,34
3,0,3074,False,0.017430,0.100254,0.064270,0.150944,252
4,0,8569,False,0.000000,0.059708,0.061508,0.148979,80
...,...,...,...,...,...,...,...,...
1072075,35735,36780,False,0.000000,0.057894,0.053736,0.100020,39
1072076,35735,37698,False,0.116613,0.000000,0.045100,0.098611,23
1072077,35735,36772,False,0.045117,0.031387,0.037053,0.095042,18
1072078,35735,36164,False,0.058774,0.011310,0.030548,0.091382,13


## Or the profile length

In [28]:
user_popularity = np.ediff1d(sps.csr_matrix(urm_rec_train).indptr)

training_dataframe["user_profile_len"] = user_popularity[
    training_dataframe["UserID"].values.astype(int)
]
training_dataframe

,UserID,ItemID,Label,ItemKNN,RP3betaICM,Hybrid,SLIM_EN,item_popularity,user_profile_len
0,0,6380,False,0.090662,0.085996,0.094839,0.223108,162,77
1,0,572,False,0.101362,0.138505,0.181966,0.209152,56,77
2,0,15454,False,0.029662,0.051473,0.057410,0.155842,34,77
3,0,3074,False,0.017430,0.100254,0.064270,0.150944,252,77
4,0,8569,False,0.000000,0.059708,0.061508,0.148979,80,77
...,...,...,...,...,...,...,...,...,...
1072075,35735,36780,False,0.000000,0.057894,0.053736,0.100020,39,19
1072076,35735,37698,False,0.116613,0.000000,0.045100,0.098611,23,19
1072077,35735,36772,False,0.045117,0.031387,0.037053,0.095042,18,19
1072078,35735,36164,False,0.058774,0.011310,0.030548,0.091382,13,19


## The same can be done with item features

It is better in this case to first create a sparse matrix replicating the rows needed and ten transform it into a sparse dataframe

**WARNING** dataframes are not sparse structures and this may cause the memory requirements to explode

In [29]:
reduced_features_df = pd.read_csv("reduced_features.csv", index_col=0)


In [30]:
training_dataframe = training_dataframe.set_index("ItemID").join(
    reduced_features_df, how="inner"
)
training_dataframe = training_dataframe.reset_index()


## Then you can train XGBoost to rerank those prediction using as lable whether they should be recommended or not

In [31]:
training_dataframe = training_dataframe.sort_values("UserID").reset_index()
training_dataframe.drop(columns=["index"], inplace=True)
training_dataframe

,ItemID,UserID,Label,ItemKNN,RP3betaICM,Hybrid,SLIM_EN,item_popularity,user_profile_len,feature_0,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,6380,0,False,0.090662,0.085996,0.094839,0.223108,162,77,0.0,...,7.914738,7.973606,7.710864,7.898441,7.890414,7.955359,9.443233,7.914578,7.905386,7.885514
1,516,0,False,0.050269,0.062577,0.089285,0.095394,16,77,0.0,...,4.604521,4.595179,4.946917,4.577198,4.588721,4.620558,5.970020,4.582166,4.552959,4.565639
2,2676,0,False,0.050311,0.089827,0.094722,0.095503,100,77,0.0,...,9.440227,9.424107,9.411556,9.401624,9.373932,9.478371,11.432402,9.413996,9.378312,9.460876
3,1700,0,False,0.062459,0.011707,0.036250,0.095702,35,77,0.0,...,6.593676,6.600882,6.415812,6.538977,6.565036,6.587192,8.271248,6.561668,6.535735,6.590301
4,22929,0,False,0.000000,0.052652,0.052662,0.096249,26,77,0.0,...,6.767047,6.826216,6.346330,6.757380,6.828654,6.770781,8.296288,6.763037,6.773791,6.799559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072075,37019,35735,True,0.098799,0.275635,0.273405,0.271843,42,19,0.0,...,0.782566,0.786711,0.833710,0.785519,0.784728,0.784749,0.813035,0.792324,0.794669,0.787533
1072076,37657,35735,False,0.183295,0.283351,0.281194,0.355287,61,19,0.0,...,3.992536,4.001902,4.449271,3.991877,3.986163,4.013656,5.036263,3.989670,4.000650,3.971535
1072077,36164,35735,False,0.058774,0.011310,0.030548,0.091382,13,19,0.0,...,2.072210,2.078412,2.354641,2.083080,2.077404,2.085098,2.056932,2.097954,2.084653,2.085045
1072078,36917,35735,False,0.053495,0.078757,0.082068,0.138379,45,19,0.0,...,0.894510,0.903181,0.934020,0.891583,0.895630,0.896680,0.910478,0.901868,0.905822,0.895701


## Target Encoding

In [32]:
from category_encoders import TargetEncoder

# Define the target encoder
encoder = TargetEncoder(cols=["UserID", "ItemID"])

# Fit the encoder on the training data
encoder.fit(training_dataframe[["UserID", "ItemID"]], training_dataframe["Label"])

# Transform the dataset
training_dataframe_encoded = training_dataframe.copy()
training_dataframe_encoded[["UserID", "ItemID"]] = encoder.transform(training_dataframe[["UserID", "ItemID"]])


/opt/miniconda3/envs/recsys/lib/python3.9/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
/opt/miniconda3/envs/recsys/lib/python3.9/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
/opt/miniconda3/envs/recsys/lib/python3.9/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .filln

In [33]:
training_dataframe_encoded

,ItemID,UserID,Label,ItemKNN,RP3betaICM,Hybrid,SLIM_EN,item_popularity,user_profile_len,feature_0,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0.050633,0.065606,False,0.090662,0.085996,0.094839,0.223108,162,77,0.0,...,7.914738,7.973606,7.710864,7.898441,7.890414,7.955359,9.443233,7.914578,7.905386,7.885514
1,0.040496,0.065606,False,0.050269,0.062577,0.089285,0.095394,16,77,0.0,...,4.604521,4.595179,4.946917,4.577198,4.588721,4.620558,5.970020,4.582166,4.552959,4.565639
2,0.038869,0.065606,False,0.050311,0.089827,0.094722,0.095503,100,77,0.0,...,9.440227,9.424107,9.411556,9.401624,9.373932,9.478371,11.432402,9.413996,9.378312,9.460876
3,0.030626,0.065606,False,0.062459,0.011707,0.036250,0.095702,35,77,0.0,...,6.593676,6.600882,6.415812,6.538977,6.565036,6.587192,8.271248,6.561668,6.535735,6.590301
4,0.008161,0.065606,False,0.000000,0.052652,0.052662,0.096249,26,77,0.0,...,6.767047,6.826216,6.346330,6.757380,6.828654,6.770781,8.296288,6.763037,6.773791,6.799559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072075,0.043620,0.163080,True,0.098799,0.275635,0.273405,0.271843,42,19,0.0,...,0.782566,0.786711,0.833710,0.785519,0.784728,0.784749,0.813035,0.792324,0.794669,0.787533
1072076,0.048809,0.163080,False,0.183295,0.283351,0.281194,0.355287,61,19,0.0,...,3.992536,4.001902,4.449271,3.991877,3.986163,4.013656,5.036263,3.989670,4.000650,3.971535
1072077,0.070869,0.163080,False,0.058774,0.011310,0.030548,0.091382,13,19,0.0,...,2.072210,2.078412,2.354641,2.083080,2.077404,2.085098,2.056932,2.097954,2.084653,2.085045
1072078,0.065527,0.163080,False,0.053495,0.078757,0.082068,0.138379,45,19,0.0,...,0.894510,0.903181,0.934020,0.891583,0.895630,0.896680,0.910478,0.901868,0.905822,0.895701


### To use the ranker one first needs to specify the size of the groups, a group is the dimension you rank on, in this case each group corresponds to a user. Since we have generated a fixed number of candidates for each user (30) all groups have the same length.

In [35]:
from xgboost import XGBRanker

# Create model with best parameters
XGB_model = XGBRanker(
    objective="rank:pairwise",
    n_estimators=100,
    learning_rate=0.1,
    reg_alpha=0.1,
    reg_lambda=0.01,
    max_depth=5,
    max_leaves=0,
    grow_policy='depthwise',
    booster='gbtree',
    random_state=42,
    verbosity=0
)

# Prepare data
y_train = training_dataframe["Label"]
X_train = training_dataframe.drop(columns=["Label"])

In [36]:
# Calculate groups (30 recommendations per group)
groups = np.bincount(np.arange(len(training_dataframe)) // 30)

# Train the model
XGB_model.fit(X_train, y_train, group=groups, verbose=True)

XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None,
          device=None, early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None,
          grow_policy='depthwise', importance_type=None,
          interaction_constraints=None, learning_rate=0.1, max_bin=None,
          max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None,
          max_depth=5, max_leaves=0, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=100,
          n_jobs=None, num_parallel_tree=None, objective='rank:pairwise', ...)

### Once the model is trained we can use it to compute predictions. Each prediction will refer to a specific user-item pair, which we will ten need to rank as we do in any other recommender model.

### Important: In order to use this model to predict the score of new datapoints (i.e., new recommendations) we have to repeat the same data processing steps but:
- We do not need a train-label split, we can user all the data we have to compute the predictions and the features
- The recommendation models used to generate the scores should be trained on all the available data

In [37]:
# Let's say I want to compute the prediction for a group of user-item pairs, for simplicity I will use a slice of the data used
# for training because it already contains all the features
X_to_predict = X_train[X_train["UserID"] == 10]

XGB_model.predict(X_to_predict)

array([-0.8242098 , -1.1867849 , -0.60873455, -0.88321245, -1.1089854 ,
       -1.0569506 , -1.1895977 , -1.0315582 , -0.8354469 , -0.7354505 ,
       -1.0770136 , -1.2832246 , -1.0614543 , -1.1531764 , -0.78594464,
       -0.9433018 , -0.81658614,  0.1906204 ,  0.35758033, -0.34596065,
        0.07846721, -0.19885851, -0.8415474 , -0.82657963, -0.6717294 ,
       -0.65372336, -0.593208  , -0.9215755 , -0.88656217, -0.45770893],
      dtype=float32)

### We can have a look to the feature importance to assess which are the most informative ones

<Axes: title={'center': 'Weight (Frequence)'}, xlabel='F score', ylabel='Features'>

In [38]:
from xgboost import plot_importance
plot_importance(XGB_model, importance_type="weight", title="Weight (Frequence)")

### Note, here ItemID and UserID are provided as integers, meaning that XGBoost will use them as any integer number and may split user groups according to whether their ID is < or > of a certain value. This makes no sense of course because the IDs are not ordinal, they are categorical and the specific numerical value of an IDs has no relation with the semantics of the problem. 

How to address this? 
- Use one-hot-encoded values -> drawback, the number of columns becomes very large
- Use the native "Categorical" data type -> drawback, it is still experimental and may not work very well
- Use another representation of the IDs, such as target encoding -> drawback, some further processing is needed and no teaching material is provided on this

In [41]:
X_train["UserID"] = X_train["UserID"].astype("category")
X_train["ItemID"] = X_train["ItemID"].astype("category")

In [ ]:
XGB_model = XGBRanker(
    objective="rank:{}".format(objective),
    n_estimators=int(n_estimators),
    random_state=random_seed,
    learning_rate=learning_rate,
    reg_alpha=reg_alpha,
    reg_lambda=reg_lambda,
    max_depth=int(max_depth),
    max_leaves=int(max_leaves),
    grow_policy=grow_policy,
    verbosity=0,  # 2 if self.verbose else 0,
    booster=booster,
    enable_categorical=True,
    tree_method="hist",  # Supported tree methods are `gpu_hist`, `approx`, and `hist`.
)

XGB_model.fit(X_train, y_train, group=groups, verbose=True)

In [ ]:
X_to_predict = X_train[X_train["UserID"] == 10]

XGB_model.predict(X_to_predict)

In [39]:
X_to_predict.shape

(30, 58)

<Axes: title={'center': 'Weight (Frequence)'}, xlabel='F score', ylabel='Features'>

In [40]:
plot_importance(XGB_model, importance_type="weight", title="Weight (Frequence)")

In [ ]:
import pandas as pd
from category_encoders import TargetEncoder

# Load the target users
target_users = pd.read_csv("./data/data_target_users_test.csv")

# Function to generate recommendations for a single user
def recommend_xgboost(user_id, model, X_train, encoder, top_k=10):
    # Filter data for the user
    X_to_predict = X_train[X_train["UserID"] == user_id].copy()
    
    # Apply the same encoding
    X_to_predict[["UserID", "ItemID"]] = encoder.transform(X_to_predict[["UserID", "ItemID"]])
    
    # Drop columns not needed for prediction
    prediction_columns = [col for col in X_train.columns if col not in ["Label", "group_id"]]
    X_to_predict = X_to_predict[prediction_columns]
    
    # Predict scores
    predicted_scores = model.predict(X_to_predict)
    
    # Get original ItemIDs before they were encoded
    original_item_ids = X_train[X_train["UserID"] == user_id]["ItemID"].values
    
    # Create DataFrame with original ItemIDs and scores
    results = pd.DataFrame({
        'ItemID': original_item_ids,
        'Score': predicted_scores
    })
    
    # Get top k recommendations using original ItemIDs
    top_items = results.sort_values("Score", ascending=False).head(top_k)
    
    return top_items["ItemID"].tolist()

# Apply the recommendation function for all target users
top_k = 10
target_users['item_list'] = target_users['user_id'].apply(
    lambda user_id: " ".join(map(str, recommend_xgboost(user_id, XGB_model, training_dataframe, encoder, top_k)))
)

# Save the recommendations to a CSV file
target_users.to_csv("submission_xgboost.csv", index=False)

### How to perform hyperparameter tuning?
The issue with this method is that you need a label which should be an item the user has not interacted with but that is a correct recommendation. In practice the idea is:
- Split the data in the usual training-validation-test
- Split the training data in two: one part you use to train the recommenders and another you use as the hidden Label to train XGBoost
- Evaluate your predictions on the validation data as you did for any other recommender model. Use this to select the optimal hyperparameters.
- Given the selected hyperparameters, train the recommender models on all the available data and use all the available data to compute the features used by XGBoost.

Challenge: Since the label we use for training XGBoost is the split of a split, it may happen that the actual correct recommendations are very few. This will result in a problem that is very unbalanced towards zero and will make the training difficult and the evaluation noisy. To mitigate this you may use k-fold cross validation and define the valdation result of a certain hyperparameter configuration as the average obtained with k different training-label splits.